In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
#under edit-> notebook settings -> select GPU

'/device:GPU:0'

In [2]:
!pip install keras

In [0]:
!pip install -U -q PyDrive

In [0]:
import sklearn.datasets as skds
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import matplotlib.pyplot as plt



In [5]:

import os
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [9]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: trials.zip, id: 1qyWYH6Geg0K8uSoNWGhz91TsxsjrgMLa
title: train.csv, id: 1GKRE9FDW8jIA5GazeIszlg9MjKZpgMiw
title: cat_dog_sub.zip, id: 1Bwf1L1TDJ5xz_iCqvdhJTAIh2zBYvLSP
title: Colab Notebooks, id: 1FBXetgaF4zoIcCX2vNqaLnkOHJklPftB


In [0]:
downloaded = drive.CreateFile({'id': '1GKRE9FDW8jIA5GazeIszlg9MjKZpgMiw'})
downloaded.GetContentFile('train.csv')

In [11]:
import os
os.listdir()

['.config', 'train.csv', 'adc.json', 'sample_data']

In [0]:
path = "train.csv"
qu = pd.read_csv(path)
# Dataset is now stored in a Pandas Dataframe

In [14]:
qu.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [0]:
qu_train,qu_test=train_test_split(qu,test_size=0.2,random_state=2)

In [0]:
x_train=qu_train['question_text']
y_train=qu_train['target']
x_test=qu_test['question_text']
y_test=qu_test['target']

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
sent_lens=[]
for sent in qu_train['question_text']:
    #print(sent)
    sent_lens.append(len(word_tokenize(sent)))

In [26]:
max(sent_lens), min(sent_lens),sent_lens[0:20]

(412,
 1,
 [13, 10, 21, 10, 9, 17, 6, 14, 16, 9, 7, 15, 13, 39, 16, 23, 20, 13, 6, 8])

In [34]:
np.quantile(sent_lens,1.0),np.quantile(sent_lens,0.95)
# We can see that 95% review text are of lengths less than or equal to 30. We'll keep the max length to 30 no change

(412.0, 30.0)

In [38]:
#max_len = 30
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(x_train)
sequences_train = tok.texts_to_sequences(x_train)
len(sequences_train),sequences_train[0:5]

(1044897,
 [[2, 3, 1, 34, 79, 5, 3375, 4, 1702, 5916, 44, 242],
  [9, 11, 14, 91, 1, 2532, 5, 1602, 3144],
  [16,
   3,
   18,
   1239,
   549,
   11352,
   10,
   697,
   111,
   1165,
   114,
   135,
   21,
   413,
   66,
   136,
   18,
   296],
  [9, 26, 1772, 377, 28, 296, 59, 4, 103],
  [9, 72, 31, 8, 311, 13, 261, 412]])

In [37]:
max_len = 30
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(x_train)
sequences_train = tok.texts_to_sequences(x_train)
len(sequences_train),sequences_train[0:5]

(1044897,
 [[2, 3, 1, 34, 79, 5, 3375, 4, 1702, 5916, 44, 242],
  [9, 11, 14, 91, 1, 2532, 5, 1602, 3144],
  [16,
   3,
   18,
   1239,
   549,
   11352,
   10,
   697,
   111,
   1165,
   114,
   135,
   21,
   413,
   66,
   136,
   18,
   296],
  [9, 26, 1772, 377, 28, 296, 59, 4, 103],
  [9, 72, 31, 8, 311, 13, 261, 412]])

In [0]:
#tok.index_word

In [41]:
len(tok.index_word)

196201

In [0]:
vocab_len=len(tok.index_word.keys())

In [44]:
vocab_len

196201

In [0]:
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

In [46]:
sequences_matrix_train

array([[   0,    0,    0, ..., 5916,   44,  242],
       [   0,    0,    0, ...,    5, 1602, 3144],
       [   0,    0,    0, ...,  136,   18,  296],
       ...,
       [   0,    0,    0, ...,  392,    7,  678],
       [   0,    0,    0, ...,    6,   28,  560],
       [   0,    0,    0, ..., 5472,    6,  142]], dtype=int32)

In [0]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(vocab_len+1,500,input_length=max_len,mask_zero=True)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [48]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

W0807 05:11:01.357714 140015657162624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 05:11:01.365007 140015657162624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 05:11:01.370324 140015657162624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0807 05:11:01.581446 140015657162624 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instruct

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 500)           98101000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                144640    
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [0]:
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [51]:
model.fit(sequences_matrix_train,y_train,batch_size=500,epochs=5,validation_data=(sequences_matrix_test,y_test))
#epochs=50 in real

Train on 1044897 samples, validate on 261225 samples
Epoch 1/5
1044897/1044897 [==============================] - 290s 277us/step - loss: 0.1201 - acc: 0.9536 - val_loss: 0.1133 - val_acc: 0.9548
Epoch 2/5
1044897/1044897 [==============================] - 290s 277us/step - loss: 0.0979 - acc: 0.9607 - val_loss: 0.1141 - val_acc: 0.9557
Epoch 3/5
1044897/1044897 [==============================] - 291s 278us/step - loss: 0.0802 - acc: 0.9674 - val_loss: 0.1243 - val_acc: 0.9544
Epoch 4/5
1044897/1044897 [==============================] - 289s 277us/step - loss: 0.0628 - acc: 0.9742 - val_loss: 0.1436 - val_acc: 0.9529
Epoch 5/5
1044897/1044897 [==============================] - 289s 277us/step - loss: 0.0471 - acc: 0.9806 - val_loss: 0.1733 - val_acc: 0.9517


In [0]:
predictions=model.predict(sequences_matrix_test)

In [0]:
from sklearn.metrics import roc_auc_score

In [54]:
roc_auc_score(y_test,predictions)

0.9306429665583046